In [5]:
# General Purpose
import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import odeint

# Jupyter Specifics
from IPython.display import HTML
from ipywidgets.widgets import interact, IntSlider, FloatSlider, Layout

%matplotlib inline

style = {'description_width': '150px'}
slider_layout = Layout(width='99%')

def main(initial_salary, savings_ratio, extraordinary_expenses, fixed_costs, days):
    saving_limit = savings_ratio * initial_salary

    def function(capital, time):
        if capital <= saving_limit:
            out_rate = 0
        else:
            out_rate = extraordinary_expenses * (capital - saving_limit)
        return -fixed_costs - out_rate

    time = np.linspace(0, days, days * 10)

    solution = odeint(function, initial_salary, time)

    #Graphic details
    fig, ax = plt.subplots(figsize=(15, 10))

    ax.plot((0, days), (saving_limit, saving_limit), label='Saving Limit')
    ax.plot(time, solution, label='Capital(t)')

    if days <= 60:
        step = 1
        rotation = "horizontal"
    elif days <= 300:
        step = 5
        rotation = "vertical"
    else:
        step = 10
        rotation = "vertical"

    ax.set_xticklabels(np.arange(0, days + 1, step, dtype=np.int), rotation=rotation)
    ax.set_xticks(np.arange(0, days + 1, step))

    ax.set_yticks(np.arange(0, initial_salary * 1.1, initial_salary / 20))

    ax.set_xlim([0, days])
    ax.set_ylim([0, initial_salary * 1.1])
    ax.set_xlabel('Days')
    ax.set_ylabel('Capital $')
    ax.legend(loc='best')
    ax.grid()

    plt.show()

interact(main, initial_salary=IntSlider(min=0, max=25000, step=500, value=15000, description='Initial Salary', style=style, layout=slider_layout),
               savings_ratio=FloatSlider(min=0, max=1, step=0.01, value=0.2, description='Savings Ratio', style=style, layout=slider_layout),
               extraordinary_expenses=FloatSlider(min=0, max=1, step=0.005, description='Extraordinary Expenses', style=style, value=0.3, layout=slider_layout),
               fixed_costs=IntSlider(min=1, max=1000, step=1, value=100, description='Fixed Costs', style=style, layout=slider_layout),
               days=IntSlider(min=1, max=600, step=5, value=30, description='Total Number of Days', style=style, layout=slider_layout)
        );


interactive(children=(IntSlider(value=15000, description='Initial Salary', layout=Layout(width='99%'), max=250…

In [3]:
# -*- coding: utf-8 -*-
"""

XXX brouillon XXX
pointillé noir = rayons correspondant à un objet sur l'axe à la position 
vérifiant les relations de conjugaison

rayons bleus : correspondent à l'objet le plus loin net
rayons rouges : correspondent à l'objet le plus proche net


Les calculs sont fait dans le cadre de l'approximation des petits angles, ne 
tient pas compte du côté non stigmatique des lentilles, en particulier aux 
grandes ouvertures. Calculs non garantis car fait vite fait sur un brouillon.
(Basé sur le formalisme de l'optique matricielle, plus facile qu'avec les 
relations de conjugaison et de la trigo)

Ne tiens pas compte non plus du phénomène de diffraction qui est le phénomène
(à ma connaissance) qui limite la résolution lorsque les diaphragmes sont très
fermés.

"""
%matplotlib qt
import numpy as np #testé avec np.__version__ 1.16.1
import matplotlib.pyplot as plt #testé avec matplotlib.__version__ 2.0.0
import PIL.Image as im#attention à la majuscule ; testé avec PIL.__version__ '5.4.1'
from matplotlib.widgets import Slider, Button#, RadioButtons
from scipy.signal import fftconvolve #testé avec scipy.__version__ 1.2.1


####### définition des constante et valeurs initiales

fp = 1 #distance focale de la lentille
xAp = 1.5 #position algébrique du capteur
epsi = 0.5 #taille(rayon) du «cercle de confusion
R = 2.3 #rayon du diaphragme

inf = float("inf")
#####   limite de l'espace  pour matplotlib  #####
xA = 1/(-1/fp+1/xAp) #position de l'image
lim = [2*xA,xAp*2,-4,4]   # xdeb xfin ydeb yfin
marge = 0.05
x_deb = lim[0]
x_fin = lim[1]
y_deb = lim[2]
y_fin = lim[3]
delta_x = x_fin - x_deb
delta_y = y_fin - y_deb
###### fin de déf des constantes

def traverse_vide(x_old,y_old,theta_old,x_new):
    """ optique matricielle 
        traversé du vide : theta inchangé, y change : ynew = yold + theta*dist
    """
    return y_old + (x_new-x_old)*theta_old #theta_new = theta_old
def traverse_lentille(y_old,theta_old,fp):
    """ optique matricielle 
        traversé d'une lentille : y inchangé, theta_new = theta_old - y/f'
    """
    theta_new = -y_old/fp + theta_old
    return theta_new #y_new = y_old

def traverse_lentille_arrière(y_new, theta_new, fp):
    """ optique matricielle 
        traversé d'une lentille : y inchangé, theta_new = theta_old - y/f'
        ici, on connait theta_new et on veut theta_old
    """
    theta_old = theta_new + y_new/fp
    return theta_old





fig = plt.figure(figsize = [10,6])

#axe optique
plt.plot([x_deb,x_fin],[0,0],'-k')#axe optique
deb_fleche = x_fin - 0.025*delta_x
altitude_fleche = delta_y * 0.025
plt.plot( [deb_fleche,x_fin,deb_fleche],[altitude_fleche,0,-altitude_fleche],'-k') # flèche

#lentille
x_O, f_p = 0, fp #une seule lentille ici
plt.plot([x_O,x_O],[0.995*y_deb,0.995*y_fin],'-k',lw=2)
if f_p<0:
    sg = -1
else:
    sg = 1
lx_fl = 0.05*delta_x
ly_fl = sg * 0.05*delta_y
plt.plot([x_O-lx_fl,x_O,x_O+lx_fl],[0.995*y_fin-ly_fl,0.995*y_fin,0.995*y_fin-ly_fl],'-k',lw=2)
plt.plot([x_O-lx_fl,x_O,x_O+lx_fl],[0.995*y_deb+ly_fl,0.995*y_deb,0.995*y_deb+ly_fl],'-k',lw=2)

#foyers
plt.plot([x_O+f_p]*2,[delta_y*0.01,-delta_y*0.01])
plt.plot([x_O-f_p]*2,[delta_y*0.01,-delta_y*0.01])
plt.axis('off')
plt.axis(lim)

#cercle de confusion ; position xAp
plot_cercleConf, = plt.plot([xAp]*2,[-epsi, epsi],lw = 2,color = "red")


#tracé du diaphragme
global diaphragme_bas, diaphragme_haut
diaphragme_bas, = plt.plot([-0.075,-0.075],[0.9*y_deb, -R],'-',lw = 5, color = [0.5,0.5,0.5])
diaphragme_haut, =plt.plot([-0.075,-0.075],[0.9*y_fin, R],'-',lw = 5, color = [0.5,0.5,0.5])


#rayons limites = rayon qui passe par le haut du diaphragme + haut et bas du cercle de confusion (et symétrique)
theta_lim_new1 = (epsi-R)/xAp
theta_lim_new2 = (-epsi-R)/xAp
theta_lim_old1 = traverse_lentille_arrière(R,theta_lim_new1,fp)
theta_lim_old2 = traverse_lentille_arrière(R,theta_lim_new2,fp)
x_old1 = -R/theta_lim_old1#point sur l'axe de départ pour le rayon
x_old2 = -R/theta_lim_old2
#XXX remarque, si x_old1>0, c'est qu'on a atteint l'hyperfocale => x_old1 réel = -inf


plotlim1_haut, = plt.plot([x_old1,0,x_fin],[0,R,traverse_vide(0,R,theta_lim_new1,x_fin)],"-r")
plotlim1_bas, = plt.plot([x_old1,0,x_fin],[0,-R,-traverse_vide(0,R,theta_lim_new1,x_fin)],"-r")

plotlim2_haut, = plt.plot([x_old2,0,x_fin],[0,R,traverse_vide(0,R,theta_lim_new2,x_fin)],"-b")
plotlim2_bas, = plt.plot([x_old2,0,x_fin],[0,-R,-traverse_vide(0,R,theta_lim_new2,x_fin)],"-b")

#netteté «parfaite»
plotnet_haut, = plt.plot([xA,0,x_fin],[0,R,traverse_vide(0,R,traverse_lentille(R,-R/xA,fp),x_fin)],"--k")
plotnet_bas, =plt.plot([xA,0,x_fin],[0,-R,-traverse_vide(0,R,traverse_lentille(R,-R/xA,fp),x_fin)],"--k")





####### création des sliders
axcolor = 'lightgoldenrodyellow'
ax_R = plt.axes([0.05, 0.05, 0.2, 0.1], facecolor=axcolor)
ax_epsi = plt.axes([0.35, 0.05, 0.2, 0.1], facecolor=axcolor)
ax_xAp = plt.axes([0.05, 0.2, 0.2, 0.1], facecolor=axcolor)

slider_R = Slider(ax_R, 'R', 0.001,0.8*y_fin, valinit=0.4*y_fin)#, valstep=1)
slider_epsi = Slider(ax_epsi, u'ε', 0.001,0.8*y_fin, valinit=0.1*y_fin)
slider_xAp = Slider(ax_xAp, "xA'", 1.01*fp,x_fin, valinit=(2*fp+x_fin)/3)



def update(osef):
    R = slider_R.val
    epsi = slider_epsi.val
    xAp = slider_xAp.val
    xA = 1/(-1/fp+1/xAp) #position de l'image
    plot_cercleConf.set_data([xAp]*2,[-epsi, epsi])
    diaphragme_bas.set_data([-0.075,-0.075],[0.9*y_deb, -R])
    diaphragme_haut.set_data([-0.075,-0.075],[0.9*y_fin, R])
    #rayons limites = rayon qui passe par le haut du diaphragme + haut et bas du cercle de confusion (et symétrique)
    theta_lim_new1 = (epsi-R)/xAp
    theta_lim_new2 = (-epsi-R)/xAp
    theta_lim_old1 = traverse_lentille_arrière(R,theta_lim_new1,fp)
    theta_lim_old2 = traverse_lentille_arrière(R,theta_lim_new2,fp)
    if theta_lim_old2 != 0:
        x_old2 = -R/theta_lim_old2#point sur l'axe de départ pour le rayon
    else:
        x_old2 = -inf
    x_old1 = -R/theta_lim_old1
    #XXX remarque, si x_old1>0, c'est qu'on a atteint l'hyperfocale => x_old1 réel = -inf
    
    if x_old2 < 0:
        plotlim2_haut.set_data([x_old2,0,x_fin],[0,R,traverse_vide(0,R,theta_lim_new2,x_fin)])
        plotlim2_bas.set_data([x_old2,0,x_fin],[0,-R,-traverse_vide(0,R,theta_lim_new2,x_fin)])
    else:
        plotlim2_haut.set_data([x_deb,0,x_fin],[R,R,traverse_vide(0,R,traverse_lentille(R,0,fp),x_fin)])
        plotlim2_bas.set_data([x_deb,0,x_fin],[-R,-R,-traverse_vide(0,R,traverse_lentille(R,0,fp),x_fin)])
    plotlim1_haut.set_data([x_old1,0,x_fin],[0,R,traverse_vide(0,R,theta_lim_new1,x_fin)])
    plotlim1_bas.set_data([x_old1,0,x_fin],[0,-R,-traverse_vide(0,R,theta_lim_new1,x_fin)])
    
    #netteté «parfaite»
    plotnet_haut.set_data([xA,0,x_fin],[0,R,traverse_vide(0,R,traverse_lentille(R,-R/xA,fp),x_fin)])
    plotnet_bas.set_data([xA,0,x_fin],[0,-R,-traverse_vide(0,R,traverse_lentille(R,-R/xA,fp),x_fin)])
    profondeur = x_old1-x_old2
    if profondeur < 0:
        print("profondeur de champ infini")
        
    else:
            
        print("profondeur de champ =",profondeur)
    if x_old2 < 0:
        print("dont",int(round(100*(x_old1-xA)/profondeur)),"% proche")
        print("et",int(round(-100*(x_old2-xA)/profondeur)),"% loin")
slider_R.on_changed(update) #connection signal/slot
slider_epsi.on_changed(update)
slider_xAp.on_changed(update)


















0

profondeur de champ = 1.9037796206139719
dont 33 % proche
et 67 % loin
profondeur de champ = 1.8557176915078297
dont 33 % proche
et 67 % loin
profondeur de champ = 1.8097863429104277
dont 33 % proche
et 67 % loin
profondeur de champ = 1.723794187562302
dont 33 % proche
et 67 % loin
profondeur de champ = 1.6834983457832007
dont 34 % proche
et 66 % loin
profondeur de champ = 1.6448610635548553
dont 34 % proche
et 66 % loin
profondeur de champ = 1.6077861895874168
dont 34 % proche
et 66 % loin
profondeur de champ = 1.4429488229002607
dont 35 % proche
et 65 % loin
profondeur de champ = 1.4135880040247661
dont 35 % proche
et 65 % loin
profondeur de champ = 1.385284907644718
dont 35 % proche
et 65 % loin
profondeur de champ = 1.3579858015695159
dont 36 % proche
et 64 % loin
profondeur de champ = 1.3316404570481981
dont 36 % proche
et 64 % loin
profondeur de champ = 1.3062018708667882
dont 36 % proche
et 64 % loin
profondeur de champ = 1.2816260133346908
dont 36 % proche
et 64 % loin
profonde

profondeur de champ = 0.9357573281658478
dont 38 % proche
et 62 % loin
profondeur de champ = 0.9222707032561901
dont 39 % proche
et 61 % loin
profondeur de champ = 0.9091339122501099
dont 39 % proche
et 61 % loin
profondeur de champ = 0.8838590266986328
dont 39 % proche
et 61 % loin
profondeur de champ = 0.8598369253890024
dont 39 % proche
et 61 % loin
profondeur de champ = 0.804709812987519
dont 40 % proche
et 60 % loin
profondeur de champ = 0.774645271924943
dont 40 % proche
et 60 % loin
profondeur de champ = 0.7376317770192411
dont 40 % proche
et 60 % loin
profondeur de champ = 0.6957112874751528
dont 40 % proche
et 60 % loin
profondeur de champ = 0.6801491782107425
dont 41 % proche
et 59 % loin
profondeur de champ = 0.6652163071350261
dont 41 % proche
et 59 % loin
profondeur de champ = 0.637097307090011
dont 41 % proche
et 59 % loin
profondeur de champ = 0.6304078526309491
dont 41 % proche
et 59 % loin
profondeur de champ = 0.6174112332537234
dont 41 % proche
et 59 % loin
profondeu

profondeur de champ = 0.9559636077125373
dont 38 % proche
et 62 % loin
profondeur de champ = 0.9444632191918958
dont 38 % proche
et 62 % loin
profondeur de champ = 0.9223097587255806
dont 38 % proche
et 62 % loin
profondeur de champ = 0.9116354236246405
dont 39 % proche
et 61 % loin
profondeur de champ = 0.8811054715269131
dont 39 % proche
et 61 % loin
profondeur de champ = 0.8713969013985361
dont 39 % proche
et 61 % loin
profondeur de champ = 0.8526330298394467
dont 39 % proche
et 61 % loin
profondeur de champ = 0.8346909365507549
dont 39 % proche
et 61 % loin
profondeur de champ = 0.8260108565961963
dont 40 % proche
et 60 % loin
profondeur de champ = 0.8092009181128987
dont 40 % proche
et 60 % loin
profondeur de champ = 0.7930855522271394
dont 40 % proche
et 60 % loin
profondeur de champ = 0.7776213861239749
dont 40 % proche
et 60 % loin
profondeur de champ = 0.7555600565434109
dont 40 % proche
et 60 % loin
profondeur de champ = 0.7347543525371412
dont 41 % proche
et 59 % loin
profon

profondeur de champ = 0.6352222491314166
dont 42 % proche
et 58 % loin
profondeur de champ = 0.6688167355664825
dont 41 % proche
et 59 % loin
profondeur de champ = 0.806256836889051
dont 40 % proche
et 60 % loin
profondeur de champ = 0.8532905686521
dont 39 % proche
et 61 % loin
profondeur de champ = 1.048738413333436
dont 37 % proche
et 63 % loin
profondeur de champ = 1.099679089450888
dont 36 % proche
et 64 % loin
profondeur de champ = 1.2720422344286035
dont 35 % proche
et 65 % loin
profondeur de champ = 1.2995763726971694
dont 34 % proche
et 66 % loin
profondeur de champ = 1.3414333074723177
dont 34 % proche
et 66 % loin
profondeur de champ = 1.355538308484493
dont 34 % proche
et 66 % loin
profondeur de champ = 1.3839842845741388
dont 34 % proche
et 66 % loin
profondeur de champ = 1.4565319416892133
dont 33 % proche
et 67 % loin
profondeur de champ = 1.5464723775866211
dont 32 % proche
et 68 % loin
profondeur de champ = 1.7535230428844981
dont 30 % proche
et 70 % loin
profondeur de

...

profondeur de champ = 9.591952362209408
dont 26 % proche
et 74 % loin
profondeur de champ = 10.452253025977505
dont 25 % proche
et 75 % loin
profondeur de champ = 11.451673542199302
dont 24 % proche
et 76 % loin
profondeur de champ = 12.624955052667847
dont 23 % proche
et 77 % loin
profondeur de champ = 14.019242590638111
dont 22 % proche
et 78 % loin
profondeur de champ = 17.761627940562402
dont 19 % proche
et 81 % loin
profondeur de champ = 20.343330460080967
dont 18 % proche
et 82 % loin
profondeur de champ = 28.071842252482732
dont 15 % proche
et 85 % loin
profondeur de champ = 34.19773311782102
dont 13 % proche
et 87 % loin
profondeur de champ = 43.248206282715245
dont 11 % proche
et 89 % loin
profondeur de champ = 57.90879209693953
dont 9 % proche
et 91 % loin
profondeur de champ = 85.5866452779219
dont 7 % proche
et 93 % loin
profondeur de champ = 156.91672229816288
dont 4 % proche
et 96 % loin
profondeur de champ = 746.9927002435007
dont 1 % proche
et 99 % loin
profondeur de ch

profondeur de champ = 0.12623801777076338
dont 49 % proche
et 51 % loin
profondeur de champ = 0.15398627063482717
dont 48 % proche
et 52 % loin
profondeur de champ = 0.16787435387205152
dont 48 % proche
et 52 % loin
profondeur de champ = 0.18177290256952316
dont 48 % proche
et 52 % loin
profondeur de champ = 0.20960489219202216
dont 48 % proche
et 52 % loin
profondeur de champ = 0.22354008713405316
dont 47 % proche
et 53 % loin
profondeur de champ = 0.2374892559417856
dont 47 % proche
et 53 % loin
profondeur de champ = 0.2514532825551168
dont 47 % proche
et 53 % loin
profondeur de champ = 0.2654330540250256
dont 47 % proche
et 53 % loin
profondeur de champ = 0.27942946070372043
dont 47 % proche
et 53 % loin
profondeur de champ = 0.2934433964362424
dont 47 % proche
et 53 % loin
profondeur de champ = 0.3074757587536312
dont 46 % proche
et 54 % loin
profondeur de champ = 0.3215274490677107
dont 46 % proche
et 54 % loin
profondeur de champ = 0.34969243991809806
dont 46 % proche
et 54 % loi

profondeur de champ = 1.1866117042143844
dont 37 % proche
et 63 % loin
profondeur de champ = 1.1955934202975278
dont 37 % proche
et 63 % loin
profondeur de champ = 1.2047198148144427
dont 37 % proche
et 63 % loin
profondeur de champ = 1.2139945588691434
dont 37 % proche
et 63 % loin
profondeur de champ = 1.2234214524062892
dont 37 % proche
et 63 % loin
profondeur de champ = 1.2330044300186913
dont 37 % proche
et 63 % loin
profondeur de champ = 1.2427475670754535
dont 37 % proche
et 63 % loin
profondeur de champ = 1.2526550861917134
dont 36 % proche
et 64 % loin
profondeur de champ = 1.262731364062577
dont 36 % proche
et 64 % loin
profondeur de champ = 1.2729809386855473
dont 36 % proche
et 64 % loin
profondeur de champ = 1.2834085169976306
dont 36 % proche
et 64 % loin
profondeur de champ = 1.294018982955361
dont 36 % proche
et 64 % loin
profondeur de champ = 1.3048174060881974
dont 36 % proche
et 64 % loin
profondeur de champ = 1.3158090505581734
dont 36 % proche
et 64 % loin
profonde

profondeur de champ = 0.9881462919090342
dont 39 % proche
et 61 % loin
profondeur de champ = 0.982113010245866
dont 39 % proche
et 61 % loin
profondeur de champ = 0.9761559306674237
dont 39 % proche
et 61 % loin
profondeur de champ = 0.9702735723498372
dont 39 % proche
et 61 % loin
profondeur de champ = 0.9644644935056255
dont 39 % proche
et 61 % loin
profondeur de champ = 0.958727290084
dont 39 % proche
et 61 % loin
profondeur de champ = 0.9474630745535146
dont 39 % proche
et 61 % loin
profondeur de champ = 0.9419334320453663
dont 40 % proche
et 60 % loin
profondeur de champ = 0.9310727510156405
dont 40 % proche
et 60 % loin
profondeur de champ = 0.9257392772090787
dont 40 % proche
et 60 % loin
profondeur de champ = 0.9204688082920216
dont 40 % proche
et 60 % loin
profondeur de champ = 0.915260201098288
dont 40 % proche
et 60 % loin
profondeur de champ = 0.8999945347832852
dont 40 % proche
et 60 % loin
profondeur de champ = 0.8950224923069514
dont 40 % proche
et 60 % loin
profondeur d

profondeur de champ = 0.527641392624719
dont 44 % proche
et 56 % loin
profondeur de champ = 0.529323077353673
dont 44 % proche
et 56 % loin
